In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

plt.rcParams['figure.figsize'] = (16, 10)
plt.rcParams['text.usetex'] = True
plt.rc('font', size=15)

In [2]:
def neural_net(features):
    layer1 = tf.sigmoid(tf.matmul(features, W1) + b1)
    layer2 = tf.sigmoid(tf.matmul(features, W2) + b2)
    layer3 = tf.concat([layer1, layer2],-1)
    layer3 = tf.reshape(layer3, shape = [-1,2])
    hypothesis = tf.sigmoid(tf.matmul(layer3, W3) + b3)
    return hypothesis

In [3]:
X = [[0, 0],
     [0, 1],
     [1, 0],
     [1, 1]]

y = [[0],
     [1],
     [1],
     [0]]

X = tf.cast(X, dtype=tf.float32)
y = tf.cast(y, dtype=tf.float32)

dataset = tf.data.Dataset.from_tensor_slices((X, y)).batch(len(X))


In [4]:
tf.random.set_seed(777)

W1 = tf.Variable(tf.random.normal([2, 1]), name='weight1')
b1 = tf.Variable(tf.random.normal([1]), name='bias1')

W2 = tf.Variable(tf.random.normal([2, 1]), name='weight2')
b2 = tf.Variable(tf.random.normal([1]), name='bias2')

W3 = tf.Variable(tf.random.normal([2, 1]), name='weight3')
b3 = tf.Variable(tf.random.normal([1]), name='bias3')

In [5]:
def loss_fn(h, y):
    cost = -tf.reduce_mean(y * tf.math.log(h) + (1 - y) * tf.math.log(1 - h))
    return cost

def grad(X, y):
    with tf.GradientTape() as tape:
        h = neural_net(X)
        losses = loss_fn(h, y)
    return tape.gradient(losses, [W1, W2, W3, b1, b2, b3])

def accuracy_fn(h, y):
    predict = tf.cast(h > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predict, y), dtype=tf.float32))
    return accuracy


In [6]:

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

for e in range(50000):
    for X, y in dataset:
        grads = grad(X, y)
        optimizer.apply_gradients(grads_and_vars=zip(grads, [W1, W2, W3, b1, b2, b3]))
        if e % 5000 == 0:
            print("Epoch: {}, Loss: {:.4f}".format(e, loss_fn(neural_net(X), y)))

Epoch: 0, Loss: 0.8487
Epoch: 5000, Loss: 0.6847
Epoch: 10000, Loss: 0.6610
Epoch: 15000, Loss: 0.6154
Epoch: 20000, Loss: 0.5722
Epoch: 25000, Loss: 0.5433
Epoch: 30000, Loss: 0.5211
Epoch: 35000, Loss: 0.4911
Epoch: 40000, Loss: 0.4416
Epoch: 45000, Loss: 0.3313


In [7]:
accuracy = accuracy_fn(neural_net(X), y)
print("Accuracy: {:.4f}".format(accuracy))

Accuracy: 1.0000
